In [ ]:
# !pip install transformers
# !pip install sentencepiece

In [ ]:
# Window os
# !pip install "git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf"
# Other os
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

In [10]:
# 데이터 준비 1
daum_movie_review_df = pd.read_csv('data-files/daum_movie_review.csv')
daum_movie_review_df.head()
daum_X = daum_movie_review_df['review'].to_list()
daum_y = daum_movie_review_df['rating'].map(lambda r: 1 if r > 5 else 0).to_list()
daum_train_X, daum_test_X, daum_train_y, daum_test_y = \
    train_test_split(daum_X, daum_y, stratify=daum_y, random_state=42)
daum_train_X, daum_valid_X, daum_train_y, daum_valid_y = \
    train_test_split(daum_train_X, daum_train_y, stratify=daum_train_y, random_state=42)

In [26]:
# 데이터 준비 1
naver_movie_train_review_df = pd.read_csv('data-files/ratings_train.txt', sep='\t')
naver_movie_test_review_df = pd.read_csv('data-files/ratings_test.txt', sep='\t')

naver_train_X = naver_movie_train_review_df['document'].to_list()
naver_train_y = naver_movie_train_review_df['label'].to_list()
naver_test_X = naver_movie_test_review_df['document'].to_list()
naver_test_y = naver_movie_test_review_df['label'].to_list()
naver_train_X, naver_valid_X, naver_train_y, naver_valid_y = \
    train_test_split(naver_train_X, naver_train_y, stratify=naver_train_y, random_state=42)

In [12]:
# daum_train_X.shape, daum_valid_X.shape, daum_test_X.shape
type(daum_train_X)

list

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

daum_train_inputs = tokenizer(daum_train_X, truncation=True, padding=True, return_tensors='pt')
daum_valid_inputs = tokenizer(daum_valid_X, truncation=True, padding=True, return_tensors='pt')
daum_test_inputs = tokenizer(daum_test_X, truncation=True, padding=True, return_tensors='pt')

In [14]:
print( daum_train_inputs['input_ids'].shape, daum_test_inputs['input_ids'].shape )

for item in daum_train_inputs.items():
    print(item[1][2].shape)
    break

torch.Size([8282, 317]) torch.Size([3682, 316])
torch.Size([317])


In [21]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        pass

    def __getitem__(self, idx):
        input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }
        input['label'] = torch.tensor(self.labels[idx])
        return input
    
    def __len__(self):
        return len(self.labels)
    
daum_train_dataset = ReviewDataset(daum_train_inputs, daum_train_y)
daum_valid_dataset = ReviewDataset(daum_valid_inputs, daum_valid_y)
daum_test_dataset = ReviewDataset(daum_test_inputs, daum_test_y)

In [22]:
for data in daum_train_dataset:
    print(data.keys())
    print(data['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])
torch.Size([317])


C:\Users\Administrator.User -2023YNCQT\AppData\Local\Temp\ipykernel_14144\2483146844.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


In [20]:
device_name = "cuda:0" if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
device

device(type='cuda', index=0)

In [24]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

metric = load_metric('accuracy')
def compute_metrics(evaluation_dataset):
    logits, labels = evaluation_dataset
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

train_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    train_dataset=daum_train_dataset,
    eval_dataset=daum_valid_dataset,
    compute_metrics=compute_metrics,
    args=train_args
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\ProgramData\miniconda3\envs\study-env2\lib\site-packages\datasets\load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/5180 [00:00<?, ?it/s]

C:\Users\Administrator.User -2023YNCQT\AppData\Local\Temp\ipykernel_14144\2483146844.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


KeyboardInterrupt: 

In [30]:
!pip list | findstr accelerate